In [0]:
import json

dbutils.widgets.text("vaultkey", "","")
dbutils.widgets.get("vaultkey") 
vaultkey = getArgument("vaultkey")

vaultkeyTier3 = vaultkey+"-Tier3"
storage_details_tier3      = dbutils.secrets.get(scope = "azure_vault", key = vaultkeyTier3)
storage_json_tier3         = json.loads(storage_details_tier3)
storage_account_name_tier3 = storage_json_tier3["Name"]

In [0]:
from pyspark.sql import *
from pyspark.sql import functions as f
from pyspark.sql.functions import mean, min, max, split
from delta.tables import *
from dateutil import parser, tz
from datetime import datetime
from dateutil.relativedelta import *
from dateutil.parser import parse
from pyspark.sql.functions import lower, col, lit

In [0]:
def PublishData(instanceCode,tablename,ViewName):  
  publish_sql = "select * from gacfreighttier2meta."+tablename+" where InstanceCode='"+instanceCode+"'"
  publish_df=spark.sql(publish_sql)
  publish_df.createOrReplaceTempView(ViewName)  

In [0]:
office_sql = 'SELECT Office.MainLocCode from Office Inner Join Individual On Individual.OfficeFk = Office.Id WHERE individual.Id = {UserId}'

local_currency_sql = "SELECT LocalCurrencyFk, Office.CurrencyDecimalPlaces FROM Office INNER JOIN Individual ON Individual.OfficeFk = Office.Id  WHERE Individual.Id = {UserId}"

dispatchedat_sql = """SELECT WarehousePackage.WarehouseOutFk, MIN(WarehousePackage.DispatchedAt) DispatchedAt
 FROM WarehousePackage 
 WHERE WarehousePackage.IsDispatched = 1 AND
    CAST(WarehousePackage.DispatchedAt as DATE) >= cast('{DispatchedDateFrom}' as date) AND
    CAST(WarehousePackage.DispatchedAt as DATE) <= cast('{DispatchedDateTo}' as date)
 GROUP BY WarehousePackage.WarehouseOutFk"""

final_sql = """
 select  concat(cast(Year(WarehousePackage.DispatchedAt) as string) , right(concat('00' , cast(Month(WarehousePackage.DispatchedAt) as string)),2)) as Month,
   PlantRig.Code as RigName,
   ModeOfShipment.Name as TransportMode,
      CASE WHEN ifnull(WarehouseOut.DispatchedAt, WarehouseDispatched.DispatchedAt) >= '20180706' THEN
           CASE WHEN BookingHouse.JobMode = 'A' AND SubProduct.IsUrgent = 1 THEN 'P1'
       WHEN BookingHouse.JobMode = 'A' AND SubProduct.IsUrgent = 0 THEN 'P2'
       ELSE 'P3'
    END
    ELSE ifnull(ModeOfShipment.Priority, 'P3')
   END AS Priority,         
   CAST(PurchaseOrder.PoNumber as string) as PoNumber,  
   ifnull(WarehousePackage.WeightKgs,0) as WeightKgs,
   ifnull(WarehousePackage.VolumeCbm,0)  as VolumeCbm,
   Currency.Code as CargoCurrency,
   Sum(ifnull(PurchaseOrderDetail.UnitPrice,0) * ifnull(WarehousePackageDetail.ReceivedQty,0)) as CargoValue,
   Origin.UnName as Origin,
   Destination.UnName as Destination,
   ifnull(BookingHouse.ShipperName,Shipper.CommonName) as Shipper,
   ifnull(BookingHouse.ConsigneeName, Consignee.CommonName) as Consignee,
   Customer.Code AS CustomerCode,
   Customer.LegalName AS CustomerName,
   Warehouse.LegalName AS Warehouse,
   WarehouseLocation.Code AS WarehouseLocationCode,
   Sum(ROUND( ifnull( (WarehousePackageDetail.ReceivedQty * ifnull(PurchaseOrderDetail.CustomsUnitPrice, PurchaseOrderDetail.UnitPrice)) , 0) , 2)) AS CargoValueOriginalCurr,
   Sum(ROUND( (IfNULL(PurchaseOrderDetail.CustomsUnitPrice, PurchaseOrderDetail.UnitPrice) * (Currency.CustomsExRate)) ,{CurrencyDecimalPlaces}) * WarehousePackageDetail.ReceivedQty)  AS CargoValueLocalCurr,
   Sum(ROUND( (IfNULL(PurchaseOrderDetail.CustomsUnitPrice, PurchaseOrderDetail.UnitPrice) * (Currency.CustomsExRate))/ {UsdExchangeRate} ,{CurrencyDecimalPlaces}) * WarehousePackageDetail.ReceivedQty)  AS CargoValueUsdCurr,{isLive} as IsLive from Booking BookingHouse
   Inner Join BookingType On BookingHouse.BookingType = BookingType.Code
   Inner Join WarehouseOut On BookingHouse.Id = WarehouseOut.BookingFk
   Inner Join WarehouseDispatched On WarehouseDispatched.WarehouseOutFk = WarehouseOut.Id
   Inner Join WarehousePackage On WarehouseOut.Id = WarehousePackage.WarehouseOutFk
   Inner Join PlantRig As PlantRig On WarehouseOut.PlantRigFk = PlantRig.Id
   Inner Join ModeOfShipment On WarehouseOut.ModeOfShipmentFk = ModeOfShipment.Id
   Inner Join WarehousePackageDetail On WarehousePackage.Id = WarehousePackageDetail.WarehousePackageFk and WarehousePackageDetail.IsInactivated = 0
   Inner Join PurchaseOrderDetail On WarehousePackageDetail.PurchaseOrderDetailFk = PurchaseOrderDetail.Id and PurchaseOrderDetail.Active = 1
   Inner Join PurchaseOrder On PurchaseOrder.Id = PurchaseOrderDetail.PurchaseOrderFk and PurchaseOrder.Active = 1
   Inner Join Party AS Customer ON Customer.Id = WarehouseOut.CustomerFk
   left Join Party AS Warehouse ON Warehouse.Id = WarehousePackage.WarehouseFk
   Left Join WarehouseLocation ON WarehousePackage.WarehouseLocationFk = WarehouseLocation.Id
   Left Join Booking BookingMaster On BookingHouse.BookingFk = BookingMaster.Id
   Left Join Location As Origin On if(BookingType.IsHouse = 1, BookingMaster.OriginLocationMainFk,BookingHouse.OriginLocationMainFk) = Origin.Id
   Left Join Location As Destination On if(BookingType.IsHouse = 1, BookingMaster.DestinationLocationMainFk,BookingHouse.DestinationLocationMainFk) = Destination.Id
   Left Join Party As Shipper On BookingHouse.ShipperFk = Shipper.Id
   Left Join Party As Consignee On BookingHouse.ConsigneeFk = Consignee.Id
   Left Join Currency As Currency On PurchaseOrderDetail.CurrencyFk = Currency.Id
   LEFT JOIN SubProduct on SubProduct.Id=BookingHouse.SubProductFk
 where PurchaseOrder.CustomerFk = {CustomerFk} and
   cast(WarehousePackage.DispatchedAt as date) >= cast('{DispatchedDateFrom}' as date) and
   cast(WarehousePackage.DispatchedAt as Date) <= cast('{DispatchedDateTo}' as date) and
   WarehousePackage.IsDispatched = 1    
 group by PlantRig.Code,
   ModeOfShipment.Name,
      CASE WHEN IfNULL(WarehouseOut.DispatchedAt, WarehouseDispatched.DispatchedAt) >= '20180706' THEN
           CASE WHEN BookingHouse.JobMode = 'A' AND SubProduct.IsUrgent = 1 THEN 'P1'
       WHEN BookingHouse.JobMode = 'A' AND SubProduct.IsUrgent = 0 THEN 'P2'
       ELSE 'P3'
    END
    ELSE IfNULL(ModeOfShipment.Priority, 'P3')
   END,
   PurchaseOrder.PoNumber,
    concat(cast(Year(WarehousePackage.DispatchedAt) as string) , right(concat('00' , cast(Month(WarehousePackage.DispatchedAt) as string)),2)),
   Currency.Code,
   Origin.UnName,
   Destination.UnName,
   ifnull(BookingHouse.ShipperName,Shipper.CommonName),
   ifnull(BookingHouse.ConsigneeName, Consignee.CommonName),
   Customer.Code,
   Customer.LegalName,
   Warehouse.LegalName,
   WarehouseLocation.Code,
   ifnull(WarehousePackage.WeightKgs,0),
   ifnull(WarehousePackage.VolumeCbm,0)

   UNION ALL

  select  concat(cast(Year(WarehousePackage.DispatchedAt) as string) , right(concat('00' , cast(Month(WarehousePackage.DispatchedAt) as string)),2)) as Month,
   PlantRig.Code as RigName,
   ModeOfShipment.Name as TransportMode,
   IfNULL(ModeOfShipment.Priority,'P3') as Priority,
   cast(PurchaseOrder.PoNumber as string) as PoNumber,  
   IfNULL(BookingHouse.GrossWeight,0) as WeightKgs,
   IfNULL(BookingHouse.Volume,0) as VolumeCbm,
   Currency.Code as CargoCurrency,
   sum(IfNULL(PurchaseOrderDetail.UnitPrice,0) * IfNULL(WarehousePackageDetail.ReceivedQty,0)) as CargoValue,
   BookingHouse.Origin as Origin,
   BookingHouse.Destination as Destination,
   BookingHouse.ShipperName as Shipper,
   BookingHouse.ConsigneeName as Consignee,
   Customer.Code AS CustomerCode,
   Customer.LegalName AS CustomerName,
   Warehouse.LegalName AS Warehouse,
   WarehouseLocation.Code AS WarehouseLocationCode,
   ROUND( IfNULL( (WarehousePackageDetail.ReceivedQty * IfNULL(PurchaseOrderDetail.CustomsUnitPrice, PurchaseOrderDetail.UnitPrice)) , 0) , 2) AS CargoValueOriginalCurr, 
   Sum(ROUND( (IfNULL(PurchaseOrderDetail.CustomsUnitPrice, PurchaseOrderDetail.UnitPrice) * (Currency.ImportExRate)) ,{CurrencyDecimalPlaces}) * WarehousePackageDetail.ReceivedQty)  AS CargoValueLocalCurr,
   Sum(ROUND( (IfNULL(PurchaseOrderDetail.CustomsUnitPrice, PurchaseOrderDetail.UnitPrice) * (Currency.ImportExRate))/ {UsdImportExchangeRate} ,{CurrencyDecimalPlaces}) * WarehousePackageDetail.ReceivedQty)  AS CargoValueUsdCurr,{isLive} as IsLive

 from LegacyJob AS BookingHouse
   Inner Join WarehouseOut On WarehouseOut.Gf2BookingNumber = BookingHouse.BookingNumber
   Inner Join WarehouseDispatched On WarehouseDispatched.WarehouseOutFk = WarehouseOut.Id
   Inner Join WarehousePackage On WarehouseOut.Id = WarehousePackage.WarehouseOutFk
   Inner Join PlantRig As PlantRig On WarehouseOut.PlantRigFk = PlantRig.Id
   Inner Join ModeOfShipment On WarehouseOut.ModeOfShipmentFk = ModeOfShipment.Id
   Inner Join WarehousePackageDetail On WarehousePackage.Id = WarehousePackageDetail.WarehousePackageFk and WarehousePackageDetail.IsInactivated = 0
   Inner Join PurchaseOrderDetail On WarehousePackageDetail.PurchaseOrderDetailFk = PurchaseOrderDetail.Id and PurchaseOrderDetail.Active = 1
   Inner Join PurchaseOrder On PurchaseOrder.Id = PurchaseOrderDetail.PurchaseOrderFk and PurchaseOrder.Active = 1 
   Inner Join Party AS Customer ON Customer.Id = WarehouseOut.CustomerFk
   Left Join Party AS Warehouse ON Warehouse.Id = WarehousePackage.WarehouseFk
   Left Join WarehouseLocation ON WarehousePackage.WarehouseLocationFk = WarehouseLocation.Id
   Left Join Currency As Currency On PurchaseOrderDetail.CurrencyFk = Currency.Id
 where PurchaseOrder.CustomerFk = {CustomerFk} and
   cast(WarehousePackage.DispatchedAt as date) >= cast('{DispatchedDateFrom}' as date) and
   cast(WarehousePackage.DispatchedAt as date) <= cast('{DispatchedDateTo}' as date) and
   WarehousePackage.IsDispatched = 1 
   
 group by PlantRig.Code,
   ModeOfShipment.Name,
   IfNULL(ModeOfShipment.Priority,'P3'),
   PurchaseOrder.PoNumber,
   BookingHouse.GrossWeight,
   BookingHouse.Volume,
   concat(cast(Year(WarehousePackage.DispatchedAt) as string) , right(concat('00' , cast(Month(WarehousePackage.DispatchedAt) as string)),2)),
   Currency.Code,
   BookingHouse.Origin,
   BookingHouse.Destination,
   BookingHouse.ShipperName,
   BookingHouse.ConsigneeName,
   Customer.Code,
   Customer.LegalName,
   Warehouse.LegalName,
   WarehouseLocation.Code,
   WarehousePackageDetail.ReceivedQty,
   PurchaseOrderDetail.UnitPrice,
   PurchaseOrderDetail.CurrencyFk,
   PurchaseOrderDetail.CustomsUnitPrice, BookingHouse.Etd
   
   
 """

fn = "select '{Office}' AS GacOffice,ReportItem.* from OutData order by month"

In [0]:
def POInventoryReportGeneration(InstanceCode,UserId,PartyId,CreatedFrom,CreatedTo,mode,islive,tablename):
  PublishData(InstanceCode,"office","Office")
  PublishData(InstanceCode,"currency","Currency")
  PublishData(InstanceCode,"individual","Individual")
  PublishData(InstanceCode,"booking","Booking")
  PublishData(InstanceCode,"jobstatus","JobStatus")
  PublishData(InstanceCode,"product","Product")
  PublishData(InstanceCode,"invoice","Invoice")
  PublishData(InstanceCode,"invoicestatus","InvoiceStatus")
  PublishData(InstanceCode,"legacyjob","LegacyJob")
  PublishData(InstanceCode,"bookingtype","BookingType")
  PublishData(InstanceCode,"bookingotherinformation","BookingOtherInformation")
  PublishData(InstanceCode,"party","Party")
  PublishData(InstanceCode,"partygroup","PartyGroup")
  PublishData(InstanceCode,"subproduct","SubProduct")
  PublishData(InstanceCode,"location","Location")
  PublishData(InstanceCode,"invoicedetail","InvoiceDetail")
  PublishData(InstanceCode,"invoiceaddon","InvoiceAddon")
  PublishData(InstanceCode,"service","Service")
  #PublishData(InstanceCode,"legacyinvoices","LegacyInvoices")
  PublishData(InstanceCode,"warehouseout","WarehouseOut")
  PublishData(InstanceCode,"warehousein","WarehouseIn")
  PublishData(InstanceCode,"warehouselocation","WarehouseLocation")
  PublishData(InstanceCode,"warehousepackage","WarehousePackage")
  PublishData(InstanceCode,"warehousepackagedetail","WarehousePackageDetail")
  PublishData(InstanceCode,"purchaseorderdetail","PurchaseOrderDetail")
  PublishData(InstanceCode,"purchaseorder","PurchaseOrder")
  PublishData(InstanceCode,"modeofshipment","ModeOfShipment")
  PublishData(InstanceCode,"plantrig","PlantRig")
  PublishData(InstanceCode,"warehouseoutloadplan","WarehouseOutLoadPlan")
  
  usdcurrency = spark.sql("SELECT  Currency.Id CurrencyId,CustomsExRate,ImportExRate FROM Currency WHERE Code = 'USD' limit 1")
  usdcurrencyid = usdcurrency.rdd.collect()[0].CurrencyId
  usdexchangeratecustoms = usdcurrency.rdd.collect()[0].CustomsExRate
  usdexchangerateimport = usdcurrency.rdd.collect()[0].ImportExRate

  localcurrency = spark.sql(local_currency_sql.format(UserId=UserId))
  PCurrencyId   = localcurrency.rdd.collect()[0].LocalCurrencyFk
  CurrencyDecimalPlaces = localcurrency.rdd.collect()[0].CurrencyDecimalPlaces
  
  officedata = spark.sql(office_sql.format(UserId=UserId))
  officecode     = officedata.rdd.collect()[0].MainLocCode
  
  warehouseopackage_dispathed = spark.sql(dispatchedat_sql.format(DispatchedDateFrom=CreatedFrom,DispatchedDateTo=CreatedTo))
  warehouseopackage_dispathed.createOrReplaceTempView("WarehouseDispatched")
  
  data= spark.sql(final_sql.format(Office =officecode,CurrencyDecimalPlaces=CurrencyDecimalPlaces,isLive=islive,UsdExchangeRate=usdexchangeratecustoms,CustomerFk=PartyId,DispatchedDateFrom=CreatedFrom,DispatchedDateTo=CreatedTo,UsdImportExchangeRate=usdexchangerateimport))
  
  
  path = "abfss://analytics-zone@" + storage_account_name_tier3 + ".dfs.core.windows.net/KPISite"

  fullpath = path+"/"+tablename

  data.write.format("delta").mode(mode).save(fullpath)

  spark.sql("CREATE DATABASE IF NOT EXISTS kpisite ")

  spark.sql("create table IF NOT EXISTS "+"kpisite"+"."
                   +tablename+" using delta location '"
                   +fullpath + "'")
  return;

In [0]:
kpipoinventory="select PartyId,InstanceCode from kpisite.globalreportconfiguration where ReportCode='POINV' and IsActive='true'"
loopdata = spark.sql(kpipoinventory)
#print(loopdata)
mode='overwrite'

todayDate = datetime.now()
yearStartDate = parse(str(todayDate.year)+'-01-01')
secondrunStartDate = yearStartDate
secondrunEndDate = parse(str(todayDate.year)+'-'+str(todayDate.month)+'-01') + relativedelta(days=-1)
if(todayDate.month==1):
  secondrunStartDate = parse(str(todayDate.year-1)+'-01-01')
  secondrunEndDate = parse(str(todayDate.year-1)+'-12-31')

for entry in loopdata.rdd.collect():
#  print(entry)
  POInventoryReportGeneration(entry.InstanceCode,1,entry.PartyId,yearStartDate,todayDate,mode,1,'PoInventory')
  mode='append'
  #GlobalCostPerMOTReportGeneration(entry.InstanceCode,1,entry.PartyId,secondrunStartDate,secondrunEndDate,mode,0,'globalcostpermot')

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o233.sql.
: org.apache.spark.sql.AnalysisException: cannot resolve '`None`' given input columns: [subproduct.ChargeWeightFactor, subproduct.InstanceCode, purchaseorderdetail.ModifiedByFk, Shipper.GacSamPartyCode, Consignee.CommodityFk, Customer.Remarks, Warehouse.Code, Warehouse.IsShipper, BookingMaster.ConsigneeContactFk, purchaseorderdetail.ExportInformationCode, BookingHouse.GtErrMsg, warehousepackagedetail.PurchaseOrderDetailFk, BookingHouse.ConsigneeArrivalNoticeRequired, warehouseout.Gf2BookingNumber, warehousepackage.SYS_CHANGE_OPERATION, purchaseorder.ProjectCode, BookingMaster.ShipperContactFk, purchaseorderdetail.GtPullDate, PlantRig.Active, warehousepackagedetail.GtTrackFlag, BookingMaster.ShipperName, PlantRig.Version, Shipper.StorageHandlingRate, bookingtype.Tier2Epoc, PlantRig.OfficeCode, Warehouse.IsConsignee, Destination.UnName, Currency.CT_Id, Shipper.IsGroupKa, warehouseout.ZoneStatus, Customer.PaymentTermFk, warehousepackage.IsPacked, warehousepackage.VerifyScan, warehouselocation.InstanceCode, Customer.PartyGroupFk, BookingMaster.BookingCreatedByFk, purchaseorder.ShipperFk, Warehouse.CommonName, Origin.UnStatus, purchaseorderdetail.OriginCountryFk, warehousepackage.SYS_CHANGE_VERSION, Currency.CurrencySymbol, Consignee.PartyGroupFk, warehousepackage.IsPicked, warehouselocation.GrossWeight, Warehouse.CallFrequency, Origin.CountryFk, warehouseout.FtzEntryNumber, warehousepackage.WareHouseInFk, purchaseorderdetail.Gf2BookingNumber, BookingMaster.NotifyFk, BookingHouse.Version, BookingHouse.ModifiedOn, warehousepackage.GtTrackFlag, Currency.MinExRate, purchaseorder.Version, purchaseorder.PoDate, Shipper.IsShipper, purchaseorder.PoReceivedAt, purchaseorder.ModifiedAt, BookingHouse.ImportBookingFk, warehousepackagedetail.WarehousePackageFk, Shipper.RegisteredCountryFk, bookingtype.IsHouse, BookingMaster.IsAtdVerified, BookingMaster.ShipperFk, BookingHouse.ConsigneeReferenceNumber, BookingHouse.ConsigneeName, subproduct.DolphinAnalysisCode, Destination.ImageHash, BookingMaster.QuotationFk, BookingHouse.ConsigneeContactName, Consignee.BusinessType, BookingMaster.NotifyName, Origin.ModifiedByFk, BookingHouse.PartyName, BookingMaster.ShipperReferenceNumber, Warehouse.LocalKamFk, purchaseorder.OfficeFk, purchaseorder.CHANGE_LOG_VERSION, Shipper.IceBreakerFk, Origin.UnIsRailTerminal, BookingHouse.CoLoaderFk, warehousepackage.FreeOrBonded, purchaseorderdetail.QtyExpected, warehousepackagedetail.partitionByDay, Consignee.IsAgent, BookingHouse.BookingType, Warehouse.BackupCsrFk, warehousepackage.TrackingJobNo, warehouseout.UniversalTrackingId, modeofshipment.Tier1Epoc, bookingtype.CT_Id, purchaseorder.partitionByMonth, warehousepackage.BreakReason, purchaseorderdetail.InstanceCode, BookingHouse.IsNotifyAddressChanged, BookingMaster.partitionByYear, warehouseout.CustomerFk, Consignee.Tier1Epoc, warehousepackage.TaricCode, BookingHouse.IncoTermFk, Shipper.TaxType, Consignee.CreatedAt, Customer.InvoiceFormat, subproduct.ProductFk, BookingMaster.PartyAddressFk, purchaseorderdetail.UniversalTrackingId, Destination.Active, purchaseorderdetail.PartReferenceText2, Warehouse.IsBillingParty, Shipper.IsAgent, warehousepackage.WidthCm, warehousepackage.OriginalWarehouseLocationFk, Shipper.TaxParty, Customer.IsConsignee, Warehouse.LogoFile, purchaseorderdetail.UnitPrice, Destination.Latitude, Shipper.LegalName, warehouselocation.SYS_CHANGE_OPERATION, purchaseor